In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Large_B-cell_Lymphoma"
cohort = "GSE243973"

# Input paths
in_trait_dir = "../../input/GEO/Large_B-cell_Lymphoma"
in_cohort_dir = "../../input/GEO/Large_B-cell_Lymphoma/GSE243973"

# Output paths
out_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/GSE243973.csv"
out_gene_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE243973.csv"
out_clinical_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/clinical_data/GSE243973.csv"
json_path = "../../output/preprocess/Large_B-cell_Lymphoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Circulating monocyte counts coupled with a 4-gene signature at leukapheresis predict survival of lymphoma patients treated with CAR T"
!Series_summary	"CD19-directed chimeric antigen receptor (CAR) T cells can induce durable remissions in relapsed/refractory large B-cell lymphomas (R/R LBCL), but 60% of patients still relapse. Biological mechanisms explaining lack of disease-response  are largely unknown. To identify mechanisms of response and survival before CAR T manufacturing in 95 R/R LBCL receiving tisagenlecleucel or axicabtagene ciloleucel, we performed phenotypic, transcriptomic and functional evaluations of leukapheresis products (LK).  Transcriptomic profiling of T cells in LK, revealed a signature composed of 4 myeloid genes able to identify patients with very short progression-free survival, highlighting the role of monocytes in CAR T therapy response. Accordingly, response and survival were negatively influenced by high circulating ab

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# Review the output from previous step:
# 1. Gene Expression Data Availability
# Based on the description, this dataset contains transcriptomic data of CD3+ T cells
# from LBCL patients and healthy controls. This indicates gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability:
# From the sample characteristics dictionary:

# For trait: Row 0 contains 'disease state' which indicates health status
trait_row = 0

# For age: No information on age is provided in the sample characteristics dictionary
age_row = None

# For gender: No information on gender is provided in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion:
# For trait: Convert from disease state to binary (healthy control = 0, large B-cell lymphoma = 1)
def convert_trait(value):
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "healthy control" in value.lower():
        return 0
    elif "large b-cell lymphoma" in value.lower():
        return 1
    else:
        return None

# Since age and gender are not available, we define placeholder conversion functions
def convert_age(value):
    return None

def convert_gender(value):
    return None

# 3. Save Metadata
# Conduct initial filtering using the validate_and_save_cohort_info function
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait data available)
if trait_row is not None and 'clinical_data' in locals():
    # Extract clinical features using the existing clinical_data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM7802550': [1.0], 'GSM7802551': [1.0], 'GSM7802552': [1.0], 'GSM7802553': [1.0], 'GSM7802554': [1.0], 'GSM7802555': [1.0], 'GSM7802556': [1.0], 'GSM7802557': [1.0], 'GSM7802558': [1.0], 'GSM7802559': [1.0], 'GSM7802560': [1.0], 'GSM7802561': [1.0], 'GSM7802562': [1.0], 'GSM7802563': [1.0], 'GSM7802564': [1.0], 'GSM7802565': [1.0], 'GSM7802566': [1.0], 'GSM7802567': [1.0], 'GSM7802568': [1.0], 'GSM7802569': [1.0], 'GSM7802570': [0.0], 'GSM7802571': [0.0], 'GSM7802572': [1.0], 'GSM7802573': [1.0], 'GSM7802574': [1.0], 'GSM7802575': [1.0], 'GSM7802576': [1.0], 'GSM7802577': [0.0], 'GSM7802578': [0.0], 'GSM7802579': [0.0], 'GSM7802580': [1.0], 'GSM7802581': [1.0], 'GSM7802582': [1.0], 'GSM7802583': [1.0], 'GSM7802584': [1.0], 'GSM7802585': [1.0], 'GSM7802586': [1.0], 'GSM7802587': [1.0], 'GSM7802588': [1.0], 'GSM7802589': [1.0], 'GSM7802590': [1.0], 'GSM7802591': [0.0], 'GSM7802592': [0.0], 'GSM7802593': [0.0], 'GSM7802594': [1.0], 'GSM7802595': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Circulating monocyte counts coupled with a 4-gene signature at leukapheresis predict survival of lymphoma patients treated with CAR T"
Line 1: !Series_geo_accession	"GSE243973"
Line 2: !Series_status	"Public on Apr 24 2024"
Line 3: !Series_submission_date	"Sep 25 2023"
Line 4: !Series_last_update_date	"Apr 25 2024"
Line 5: !Series_pubmed_id	"38359407"
Line 6: !Series_summary	"CD19-directed chimeric antigen receptor (CAR) T cells can induce durable remissions in relapsed/refractory large B-cell lymphomas (R/R LBCL), but 60% of patients still relapse. Biological mechanisms explaining lack of disease-response  are largely unknown. To identify mechanisms of response and survival before CAR T manufacturing in 95 R/R LBCL receiving tisagenlecleucel or axicabtagene ciloleucel, we performed phenotypic, transcriptomic and functional evaluations of leukapheresis products (LK).  Transcriptomic profiling of T cells in LK, revealed a signatu

### Step 4: Gene Identifier Review

In [5]:
# Review gene identifiers
# Looking at the gene IDs from the output: ABCF1, ACACA, ACAD10, ACADVL, etc.
# These appear to be standard human gene symbols. They follow the official gene symbol nomenclature
# with uppercase letters and occasionally numbers. They match known human genes.
# Examples:
# - ABCF1: ATP Binding Cassette Subfamily F Member 1
# - ACACA: Acetyl-CoA Carboxylase Alpha
# - AKT1: AKT Serine/Threonine Kinase 1
# - ADAR: Adenosine Deaminase RNA Specific

# The data appears to be from the nCounter 780 gene CAR-T characterization panel,
# which typically uses standard gene symbols.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the obtained gene expression data
print("\nNormalizing gene symbols...")
# Get the gene data from previous step
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_data = get_genetic_data(matrix_file)

# Use the normalize_gene_symbols_in_index function to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: {len(normalized_gene_data.index)} genes")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data that was already saved in step 2
print("\nLoading previously saved clinical data...")
if os.path.exists(out_clinical_data_file):
    clinical_df = pd.read_csv(out_clinical_data_file)
    is_trait_available = True
    print(f"Loaded clinical data with shape: {clinical_df.shape}")
else:
    print("Clinical data file not found. Attempting to recreate it.")
    # Re-extract clinical features using correct parameters from Step 2
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Use the correct parameters from Step 2
    trait_row = 0
    
    def convert_trait(value):
        if pd.isna(value) or not isinstance(value, str):
            return None
        
        # Extract value after colon if present
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        if "healthy control" in value.lower():
            return 0
        elif "large b-cell lymphoma" in value.lower():
            return 1
        else:
            return None
    
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=None,
        convert_age=None,
        gender_row=None,
        convert_gender=None
    )
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Re-created clinical data saved to {out_clinical_data_file}")
    
    clinical_df = selected_clinical_df
    is_trait_available = True if trait_row is not None else False

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    if is_trait_available and not clinical_df.empty:
        # Convert clinical_df to format expected by geo_link_clinical_genetic_data
        # If clinical_df is already in the right format, just use it directly
        if not isinstance(clinical_df.index, pd.Index) or clinical_df.index.name != 'Large_B-cell_Lymphoma':
            # If we loaded from CSV, we need to transpose and set the index
            # First check if the trait is in the columns
            if trait in clinical_df.columns:
                # Just use the dataframe directly - it's already in the right format
                pass
            else:
                # Convert to the right format: transpose dataframe
                clinical_df_transposed = clinical_df.set_index(clinical_df.columns[0]).T
                clinical_df = clinical_df_transposed

        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
        print(f"Created linked data with {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    else:
        print("Cannot link data: clinical data is not available")
        linked_data = pd.DataFrame()
        is_trait_available = False
except Exception as e:
    print(f"Error linking clinical and genetic data: {e}")
    is_trait_available = False
    linked_data = pd.DataFrame()

# 4. Handle missing values in the linked data
if is_trait_available and not linked_data.empty:
    print("\nHandling missing values...")
    try:
        # Rename the first column to the trait name for consistency
        if linked_data.columns[0] != trait:
            linked_data = linked_data.rename(columns={linked_data.columns[0]: trait})
        
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        
    # 5. Determine whether the trait and demographic features are biased
    print("\nEvaluating feature bias...")
    try:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Trait bias determination: {is_biased}")
        print(f"Final linked data shape: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error evaluating feature bias: {e}")
        is_biased = True
else:
    print("\nSkipping missing value handling and bias evaluation as linked data is not available")
    is_biased = True

# 6. Validate and save cohort information
print("\nPerforming final validation...")
note = ""
if not is_trait_available:
    note = "Dataset does not contain required trait information"
elif is_biased:
    note = "Dataset has severe bias in the trait distribution"

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. Save the linked data if usable
print(f"\nDataset usability for {trait} association studies: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Final linked data saved to {out_data_file}")
else:
    if note:
        print(f"Reason: {note}")
    else:
        print("Dataset does not meet quality criteria for the specified trait")


Normalizing gene symbols...
After normalization: 762 genes
Normalized gene data saved to ../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE243973.csv

Loading previously saved clinical data...
Loaded clinical data with shape: (1, 85)

Linking clinical and genetic data...
Created linked data with 86 samples and 846 features

Handling missing values...
After handling missing values: 1 samples and 84 features

Evaluating feature bias...
Quartiles for 'Large_B-cell_Lymphoma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Large_B-cell_Lymphoma' in this dataset is severely biased.

Trait bias determination: True
Final linked data shape: 1 samples and 84 features

Performing final validation...

Dataset usability for Large_B-cell_Lymphoma association studies: False
Reason: Dataset has severe bias in the trait distribution
